<a href="https://colab.research.google.com/github/prathameshparit/LoRA_Image_Preprocessor/blob/main/LoRA_Image_Preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Censor

In [ ]:
!git clone https://github.com/prathameshparit/Face_Censor.git

Cloning into 'Face_Censor'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 0), reused 10 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 20.55 MiB | 33.62 MiB/s, done.


In [ ]:
%cd Face_Censor

/content/Face_Censor


In [ ]:
import os
import cv2
import time
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
class Detector:
    def __init__(self, model_path, name=""):
        self.graph = tf.Graph()
        self.model_path = model_path
        self.model_name = name
        self.sess = tf.compat.v1.Session(graph=self.graph)
        with self.graph.as_default():
            self.graph_def = tf.compat.v1.GraphDef()
            with tf.io.gfile.GFile(model_path, 'rb') as f:
                self.graph_def.ParseFromString(f.read())
                tf.import_graph_def(self.graph_def, name='')
        print(f"{self.model_name} model is created..")

    def detect_objects(self, img, threshold=0.3):
        print(
            "{} : Object detection has started..".format(self.model_name))

        start_time = time.time()
        objects = []

        # start the session
        with tf.compat.v1.Session(graph=self.graph) as sess:

            # reshpae input image to give it to the network
            rows = img.shape[0]
            cols = img.shape[1]
            image_np_expanded = np.expand_dims(img, axis=0)

            # run the model
            (num, scores, boxes,
                classes) = self.sess.run(
                    [self.sess.graph.get_tensor_by_name('num_detections:0'),
                     self.sess.graph.get_tensor_by_name('detection_scores:0'),
                     self.sess.graph.get_tensor_by_name('detection_boxes:0'),
                     self.sess.graph.get_tensor_by_name('detection_classes:0')],
                feed_dict={'image_tensor:0': image_np_expanded})

            # parse the results
            for i in range(int(num)):
                score = float(scores[0, i])
                if score > threshold:
                    obj = {}
                    obj["id"] = int(classes[0, i])
                    obj["score"] = score
                    bbox = [float(v) for v in boxes[0, i]]
                    obj["x1"] = int(bbox[1] * cols)
                    obj["y1"] = int(bbox[0] * rows)
                    obj["x2"] = int(bbox[3] * cols)
                    obj["y2"] = int(bbox[2] * rows)
                    objects.append(obj)

            print(f"{self.model_name} : {len(objects)} objects have been found ")
        end_time = time.time()
        print("{} : Elapsed time: {}".format(
            self.model_name, str(end_time - start_time)))

        return objects

In [ ]:
def overlay_oval_boxes(image, boxes):
    for box in boxes:
        x1, y1 = box["x1"], box["y1"]
        x2, y2 = box["x2"], box["y2"]
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        axes = ((x2 - x1) // 2, (y2 - y1) // 2)
        overlay = image.copy()
        cv2.ellipse(overlay, center, axes, angle=0, startAngle=0, endAngle=360, color=(0, 0, 0), thickness=cv2.FILLED)
        image = cv2.addWeighted(overlay, 1, image, 0, 0)
    return image

def process_images(input_folder, output_folder, detector, threshold):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_image_path = os.path.join(input_folder, filename)
            output_image_path = os.path.join(output_folder, filename)

            image = cv2.imread(input_image_path)
            faces = detector.detect_objects(image, threshold=threshold)
            image = overlay_oval_boxes(image, faces)

            cv2.imwrite(output_image_path, image)
            print(f'Image {filename} processed and saved successfully at {output_image_path}')

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(filename='face_censor.log', level=logging.INFO)

    input_folder_path = "train"
    output_folder_path = "output"
    model_path = "/content/Face_Censor/models/face.pb"
    threshold = 0.5

    detector = Detector(model_path=model_path, name="detection")

    try:
        # Create the output folder if it doesn't exist
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)

        process_images(input_folder_path, output_folder_path, detector, threshold)
    except Exception as e:
        logging.error(f"An error occurred: {e}")


detection model is created..
detection : Object detection has started..
detection : 1 objects have been found 
detection : Elapsed time: 1.4088878631591797
Image s-ku769pnk-mokshi-original-imagk5mfkfg83dpg-removebg-preview.png processed and saved successfully at output/s-ku769pnk-mokshi-original-imagk5mfkfg83dpg-removebg-preview.png
detection : Object detection has started..
detection : 1 objects have been found 
detection : Elapsed time: 0.14954185485839844
Image s-ku769pnk-mokshi-original-imagk5mfhusterth-removebg-preview.png processed and saved successfully at output/s-ku769pnk-mokshi-original-imagk5mfhusterth-removebg-preview.png
detection : Object detection has started..
detection : 1 objects have been found 
detection : Elapsed time: 0.14715862274169922
Image s-ku769pnk-mokshi-original-imagk5mfdgnfewzg-removebg-preview.png processed and saved successfully at output/s-ku769pnk-mokshi-original-imagk5mfdgnfewzg-removebg-preview.png
detection : Object detection has started..
detectio

# Background Remover

In [ ]:
!git clone https://huggingface.co/spaces/aryadytm/remove-photo-background/ /content/remove-photo-background
!pip install streamlit
%cd /content/remove-photo-background

In [ ]:
import os
from PIL import Image
from src.utils import change_background, matte
from src.st_style import apply_prod_style

def load_image(image_path):
    return Image.open(image_path)

def save_image(image, output_path):
    image.save(output_path)

def remove_background(input_image, output_path, background_color="#FFFFFF", alpha=1.0):
    img_matte = matte(input_image)
    img_output = change_background(input_image, img_matte, background_alpha=alpha, background_hex=background_color)
    save_image(img_output, output_path)
    return img_output

def process_image_pipeline(input_image_path, output_folder, background_color="#FFFFFF", alpha=1.0):
    # Step 1: Load the input image
    input_image = load_image(input_image_path)

    # Step 2: Remove background
    removed_bg_image = remove_background(input_image, os.path.join(output_folder, "removed_bg_image.png"), background_color, alpha)


if __name__ == "__main__":
    input_image_path = "/content/WhatsApp Image 2023-11-24 at 23.07.45_ccdea316.jpg"
    output_folder_path = "/content/output"
    background_color = "#FFFFFF"
    alpha = 1.0

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    process_image_pipeline(input_image_path, output_folder_path, background_color, alpha)
